In [1]:
!pip install py2neo
!pip install neomodel
!pip install pandas

  Using cached py2neo-2021.2.3-py2.py3-none-any.whl (177 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl (8.2 kB)
  Using cached interchange-2021.0.4-py2.py3-none-any.whl (28 kB)
     |████████████████████████████████| 138 kB 5.5 MB/s eta 0:00:01
  Using cached pansi-2020.7.3-py2.py3-none-any.whl (10 kB)
     |████████████████████████████████| 503 kB 14.8 MB/s eta 0:00:01
  Using cached neomodel-4.0.8-py3-none-any.whl (46 kB)
     |████████████████████████████████| 1.0 MB 4.6 MB/s eta 0:00:01
  Using cached neobolt-1.7.17.tar.gz (183 kB)
  Using cached neo4j-driver-4.3.6.tar.gz (76 kB)
  Created wheel for neo4j-driver: filename=neo4j_driver-4.3.6-py3-none-any.whl size=100614 sha256=35243d23e9a690eb36e12d09223f43a0998f300de95c56da31967b633f87af7a
  Stored in directory: /home/alan/.cache/pip/wheels/1e/e2/40/4186acdee3569834fac4f80fab5a1ff256523549628e15dc28
  Created wheel for neobolt: filename=neobolt-1.7.17-cp37-cp37m-linux_x86_64.whl size=695225 sha256=e5cdec55f5ab8361b5ec105b1c

In [1]:
from DataSetupPackage import DataSetUpPackage
from DataAccessLayer import DataAccessLayer
import json

In [3]:
dbcon = DataAccessLayer(username='neo4j',password='neo123456').CreateDBConnection
dbcon.ClearCurrentDB  # This will clean up the neo4jDB
DataSetUpPackage().ReadCSVAndPopulateDB.PopulateNeo4j(dbcon.Session, verbose=True)

create Disease node with name as Fin or tail rot
Set props on Disease-Fin or tail rot:
environment - ALL
affectfish - nan
cause - The most common causes of fin rot are poor water quality and improperly-low water temperature. 
Overcrowding the tank, feeding outdated food, overfeeding the fish, and moving or handling, bullying from other fish can also cause stress that leads to fin rot.
treatment - Several antibiotics are effective in treating fin rot, but the root cause must be addressed to ensure the disease doesnât return.
Treatment should include a water change and careful examination of the aquarium conditions. 
If there is food debris, vacuum the gravel and take care to avoid overfeeding in the future.
Start putting dates on your fish food, as it loses the vitamin content fairly quickly after the food container is opened. 
Feeding your fish fresh, high-quality food in smaller quantities is far better than frequent, large feedings of stale foods.
Check the pH and temperature of th

### Visualize created knowledge graph at: http://localhost:7474/browser/

* Username: neo4j

* Password: ai-user

In [3]:
dbcon = DataAccessLayer(username='neo4j',password='ai-user').CreateDBConnection
result2 = dbcon.GetAllNodeListOfType('Disease')
for resulti in result2:
    print("Disease Name: " + resulti.name)
    print(resulti.environment)
    for r in resulti.symptoms:
        print(r.name + " # " +str(r.id) +" === " + str(resulti.symptoms.relationship(r).weight) + str(resulti.symptoms.relationship(r).penalty))
    print("----------------------------")

Disease Name: Ammonia Poisoning
ALL
INFGILL # 6 === 1.0-0.2
GASPING # 225 === 1.0-0.2
----------------------------
Disease Name: Anchor Worms
Freshwater
WHITETHREAD # 12 === 1.0-0.2
WORMBODY # 11 === 1.0-0.2
INFSCALE # 10 === 1.0-0.2
FLASHING # 237 === 1.0-0.2
----------------------------
Disease Name: CO2 Poisoning
ALL
SURFACEDWELL # 248 === 1.0-0.2
GASPING # 225 === 1.0-0.2
RAPIDBREATH # 172 === 1.0-0.2
----------------------------
Disease Name: Constipation
ALL
ERSWIM # 226 === 1.0-0.2
SWIMDIFF # 156 === 1.0-0.2
BLOAT # 207 === 1.0-0.2
----------------------------
Disease Name: Fish Lice
ALL
DOTFIN # 22 === 1.0-0.2
GREYDOT # 21 === 1.0-0.2
LETHARGIC # 210 === 1.0-0.2
PALESKIN # 216 === 1.0-0.2
FLASHING # 237 === 1.0-0.2
SKINULCER # 167 === 1.0-0.2
PALEGILL # 227 === 1.0-0.2
DOTBODY # 20 === 1.0-0.2
----------------------------
Disease Name: Flukes
ALL
SUDDENDEATH # 2 === 1.0-0.2
LOSSAPPETITE # 157 === 1.0-0.2
LETHARGIC # 210 === 1.0-0.2
PALESKIN # 216 === 1.0-0.2
LOSSSCALE # 30 === 

In [4]:
newCase = dbcon.CreateOrGetCaseNode('case1')
print(newCase.name + " # " + str(newCase.id))

case1 # 105


In [5]:
symptom1 = dbcon.GetOneSymptomNode("WORMBODY")
print(symptom1.id)

11


In [6]:
dbcon = DataAccessLayer(username='neo4j',password='ai-user').CreateDBConnection
newCase = dbcon.CreateOrGetCaseNode('case1',None,True)
symptom1 = dbcon.GetOneSymptomNode("WORMBODY")
newCase = dbcon.UpdateSymptomToCase(newCase, symptom1, 0.5)
print(len(newCase.suspected_symptoms))
for r in newCase.suspected_symptoms:
    print(r.name + " # " +str(r.id) +" === " + str(newCase.suspected_symptoms.relationship(r).suspectedLevel))

1
WORMBODY # 11 === 0.5


In [7]:
for d in symptom1.diseases:
    dbcon.UpdateDiseaseToCase(newCase, d, 1/len(d.symptoms))
    print(d.name)

Anchor Worms


In [8]:
symptom2 = dbcon.GetOneSymptomNode("GASPING")
newCase = dbcon.UpdateSymptomToCase(newCase, symptom2)
for d in symptom2.diseases:
    confidence = 1/len(d.symptoms)
    rel = newCase.suspected_diseases.relationship(d)
    if rel:
        confidence += rel.confidence
    dbcon.UpdateDiseaseToCase(newCase, d, confidence)
    print(d.name + " # " + str(confidence))

Gill Mite # 0.16666666666666666
CO2 Poisoning # 0.3333333333333333
Ammonia Poisoning # 0.5
Swim Bladder Disease # 0.125
Viral Hemorrhagic SepticemiaÂ  # 0.1111111111111111
